In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException    
from datetime import date, timedelta
import time
import re
from user_agent import generate_user_agent

In [4]:
headers = {'User-Agent': generate_user_agent(device_type="desktop", os=('mac', 'linux'))}
opts = webdriver.ChromeOptions()
opts.add_argument({'User-Agent': generate_user_agent(device_type="desktop", os=('mac', 'linux'))})

In [21]:
driver = webdriver.Chrome()

In [ ]:
driver.get('https://www.google.com')

# Find search box, search for term
search_box = driver.find_element_by_name('q')
search_box.send_keys('Flüchtlinge\n')


tools = driver.find_element_by_xpath('//*[@id="hdtb-tls"]')
tools.click()
time.sleep(4)

# setting up the timeframe I want my results to be in
# starts january 1st, 2014, takes one week, 
# scrapes all sites, all results, grabs the links
# once there is no 'next'-page to click on
# clears the dates, and moves on to the next week

start = date(2014, 1, 1)
week = timedelta(days=7)
next_day = timedelta(days=1)
end = date(2014, 12, 31)
scrape_start_date = start
ending = scrape_start_date+week

# Empty list to store the links in. 
links = []

# This while loop tells Selenium to scrape until the end of the year 2014
# on a week-by-week basis. 

while scrape_start_date < end:
    print('---new week ---')
    start_time = str(scrape_start_date.strftime('%m/%d/%Y'))
    end_time = str(ending.strftime('%m/%d/%Y'))
    print(start_time)
    print(end_time)
    
    find_time = driver.find_elements_by_css_selector('div.mn-hd-txt')
    for find in find_time:
        if "Zeit" in find.text:
            find.click()
        elif "2014" in find.text:
            find.click()
            
    time.sleep(1)
    set_time = driver.find_element_by_css_selector('span#cdrlnk')
    set_time.click()
    time_begin = driver.find_element_by_css_selector('input#cdr_min')
    time_end = driver.find_element_by_css_selector('input#cdr_max')

    time_begin.clear()
    time_end.clear()
    time.sleep(0.5)
    time_begin.send_keys(start_time)

    time_end.send_keys(end_time)
    time.sleep(1)

    # Selenium now knows what week I'm looking for. 
    # Time to click the button, start the search.

    begin_search = driver.find_elements_by_class_name("cdr_go")[1]
    begin_search.click()

#    urls = driver.find_elements_by_css_selector('h3.r a')
#    for url in urls:
#        links.append(url.get_attribute('href'))
    next_page = driver.find_element_by_css_selector('#pnnext')

    # while in the second while loop, I want the results to be in a list
    # this gives me the chance to later look at specific weeks, if I want to.
    # after one week is over, and new dates are set, list gets emptied.
    
    week_link = []
    while next_page != None:
        try:
            next_page = driver.find_element_by_css_selector('#pnnext')
            next_page.click()
            time.sleep(5)
            urls = driver.find_elements_by_css_selector('h3.r a')
            for url in urls:
                week_link.append(url.get_attribute('href'))

        # if there is no "next"-page, the code breaks, and continues 
        # to run the original while loop it is nested in
        
        except NoSuchElementException:
            print('Done with this. Date is', end_time)
            break
    
    scrape_start_date = ending + next_day
    ending = scrape_start_date + week
    links.append(week_link)

In [20]:
len(links)

5

In [ ]:
df = pd.DataFrame(links)

In [45]:
df.rename(columns={0: "links"}, inplace=True)
df.head(2)

,links
0,https://www.zeit.de/gesellschaft/zeitgeschehen...
1,https://www.op-online.de/region/neu-isenburg/f...
